Load the database Social-Network.

In [2]:
%load_ext sql
%sql sqlite:///Social-Network.db

'Connected: @Social-Network.db'

**Assignment II Description**

Students at your hometown high school have decided to organize their social network using databases. So far, they have
collected information about sixteen students in four grades, 9-12. Here's the schema:

Highschooler ( ID, name, grade )
English: There is a high school student with unique _ID_ and a given _first name_ in a certain _grade_.

Friend ( ID1, ID2 )
English: The student with _ID1_ is friends with the student with _ID2_. Friendship is mutual, so if (123, 456) is in the Friend table, so is (456, 123).

Likes ( ID1, ID2 )
English: The student with _ID1_ likes the student with _ID2_. Liking someone is not necessarily mutual, so if (123, 456) is in the Likes table, there is no guarantee that (456, 123) is also present.

Your queries will run over a small data set conforming to the schema. You can view the database in the pdf file.

For your convenience, here is a graph showing the various connections between the students in our database. 9th graders
are blue, 10th graders are green, 11th graders are yellow, and 12th graders are purple. Undirected black edges indicate
friendships, and directed red edges indicate that one student likes another student.

![title](image.png)

**Instructions**

Each problem asks you to write a query in SQL. You can run your code by clicking Run button on the top of the page.

**Important Notes**

1- Your queries are executed using SQLite, so you must conform to the SQL constructs supported by SQLite.

2- Unless a specific result ordering is asked for, you can return the result rows in any order.

3- You are to translate the English into a SQL query that computes the desired result over all possible databases. I will give you the correct output two weeks later as a reference. However, just to check the answer is not enough. This means for our small sample database, even if your answer is correct, it is possible that your query does not correctly reflect the problem at hand. (For example, if we ask for a complex condition that requires accessing all of the tables, but over our small data set in the end the condition is satisfied only by Star Wars, then the query "select title from Movie where title = 'Star Wars'" will get correct answer even though it doesn't reflect the actual question.) Circumventing the system in this fashion is not a good idea to help you learn SQL. On the other hand, an incorrect attempt at a general solution is unlikely to produce the right answer, so you shouldn't be led astray by just checking the answer.

**Assignment Questions**

This assignment includes three exercises **[Social-Network Query Exercises], [SQL Social-Network Query Exercises Extras]** and **[Social-Network Modification Exercises]**. Finish all questions listed below and test if your answers are correct with reference output. Then fill your answers in this template.

**What to hand in:**
1. Generate a HTML report from this notebook template with answers (File > Download as > HTML).
2. This "Assignment_2.ipynb" file with answers.

Social-Network Query Exercises, Q1:

Find the names of all students who are friends with someone named Gabriel.

In [3]:
%%sql

SELECT Highschooler.name
FROM Highschooler, Friend, (SELECT Highschooler.ID as ID
	FROM Highschooler
	WHERE name = 'Gabriel') as gabriel
WHERE Highschooler.ID = Friend.ID1
AND Friend.ID2 = gabriel.ID

 * sqlite:///Social-Network.db
Done.


name
Jordan
Cassandra
Andrew
Alexis
Jessica


Social-Network Query Exercises, Q2:

For every student who likes someone 2 or more grades younger than themselves, return that student's name and grade, and
the name and grade of the student they like.

In [ ]:
%%sql

SELECT Highschooler.name, Highschooler.grade , student2.name, student2.grade
FROM Highschooler, Likes, (SELECT Highschooler.ID as ID, Highschooler.grade as grade, Highschooler.name as name
	FROM Highschooler) as student2
WHERE Highschooler.ID = Likes.ID1
AND student2.ID = Likes.ID2
AND (Highschooler.grade - student2.grade) >=2

Social-Network Query Exercises, Q3:

For every pair of students who both like each other, return the name and grade of both students. Include each pair only once,
with the two names in alphabetical order.

In [7]:
%%sql

SELECT Highschooler.name, Highschooler.grade , student2.name, student2.grade
FROM Highschooler, Likes, (SELECT Highschooler.ID as ID, Highschooler.grade as grade, Highschooler.name as name
	FROM Highschooler) as student2, (SELECT * FROM Likes) as Likes2
WHERE Likes.ID1 = Likes2.ID2
AND Likes.ID2 = Likes2.ID1
AND Highschooler.ID = Likes.ID1
AND student2.ID = Likes.ID2
AND  student2.name > Highschooler.name
ORDER BY Highschooler.name

 * sqlite:///Social-Network.db
Done.


name,grade,name_1,grade_1
Cassandra,9,Gabriel,9
Jessica,11,Kyle,12


Social-Network Query Exercises, Q4:

Find all students who do not appear in the Likes table (as a student who likes or is liked) and return their names and grades.
Sort by grade, then by name within each grade.

In [6]:
%%sql

SELECT Highschooler.name, Highschooler.grade
FROM Highschooler, Likes
WHERE Highschooler.ID NOT IN (SELECT Likes.ID1 FROM Likes)
AND Highschooler.ID NOT IN (SELECT Likes.ID2 FROM Likes)
GROUP BY Highschooler.name
ORDER BY Highschooler.grade, Highschooler.name

 * sqlite:///Social-Network.db
Done.


name,grade
Jordan,9
Tiffany,9
Logan,12


Social-Network Query Exercises, Q5:

For every situation where student A likes student B, but we have no information about whom B likes (that is, B does not
appear as an ID1 in the Likes table), return A and B's names and grades.

In [8]:
%%sql

SELECT Highschooler.name,Highschooler.grade, student2.name, student2.grade
FROM Highschooler, Likes, 
	(SELECT Highschooler.ID as ID, Highschooler.name as name, Highschooler.grade as grade
	FROM Highschooler) as student2

WHERE Highschooler.ID != student2.ID
AND Highschooler.ID = Likes.ID1
AND student2.ID = Likes.ID2
AND student2.ID NOT IN (SELECT Likes.ID1 FROM Likes)
ORDER BY student2.name, Highschooler.name

 * sqlite:///Social-Network.db
Done.


name,grade,name_1,grade_1
John,12,Haley,10
Austin,11,Jordan,12
Alexis,11,Kris,10
Brittany,10,Kris,10


Social-Network Query Exercises, Q6:

Find names and grades of students who only have friends in the same grade. Return the result sorted by grade, then by name
within each grade.

In [10]:
%%sql

WITH

student1 as (SELECT Highschooler.name, Highschooler.grade, Highschooler.ID
FROM Highschooler),
student2 as(SELECT Highschooler.name, Highschooler.grade, Highschooler.ID
FROM Highschooler),
 notsamegrade as (
 SELECT DISTINCT student1.ID
FROM student1, student2, Friend
WHERE student1.ID != student2.ID
AND student1.ID = Friend.ID1
AND student2.ID = Friend.ID2
AND student1.grade != student2.grade
ORDER BY student1.ID)

SELECT student1.name, student1.grade
FROM student1
WHERE student1.ID NOT IN (SELECT ID FROM notsamegrade)
ORDER BY student1.grade, student1.name



 * sqlite:///Social-Network.db
Done.


name,grade
Jordan,9
Brittany,10
Haley,10
Kris,10
Gabriel,11
John,12
Logan,12


Social-Network Query Exercises, Q7:

For each student A who likes a student B where the two are not friends, find if they have a friend C in common (who can
introduce them!). For all such trios, return the name and grade of A, B, and C.

In [11]:
%%sql

WITH

student1 as (SELECT Highschooler.name as name, Highschooler.grade as grade, Highschooler.ID as ID
FROM Highschooler),
student2 as(SELECT Highschooler.name as name, Highschooler.grade as grade, Highschooler.ID as ID
FROM Highschooler),
commonfriend as(SELECT Highschooler.name as name, Highschooler.grade as grade, Highschooler.ID as ID
FROM Highschooler),
friendlist2 as (SELECT * FROM Friend)

SELECT student1.name, student1.grade,  student2.name, student2.grade, commonfriend.name, commonfriend.grade
FROM student1, student2, commonfriend, Friend, friendlist2, Likes
WHERE student1.ID = Likes.ID1
AND student2.ID = Likes.ID2

AND student1.ID = Friend.ID1
AND commonfriend.ID = Friend.ID2

AND student2.ID = friendlist2.ID1
AND commonfriend.ID = friendlist2.ID2

AND (student1.ID, student2.ID) NOT IN Friend

 * sqlite:///Social-Network.db
Done.


name,grade,name_1,grade_1,name_2,grade_2
Andrew,10,Cassandra,9,Gabriel,9
Austin,11,Jordan,12,Andrew,10
Austin,11,Jordan,12,Kyle,12


Social-Network Query Exercises, Q8:

Find the difference between the number of students in the school and the number of different first names.

In [12]:
%%sql

WITH 
students as (SELECT count(*) as counter
FROM Highschooler),
names as (SELECT count(DISTINCT name) as counter
FROM Highschooler)

SELECT (students.counter - names.counter) as difference
FROM students, names

 * sqlite:///Social-Network.db
Done.


difference
2


Social-Network Query Exercises, Q9:

Find the name and grade of all students who are liked by more than one other student.

In [13]:
%%sql

WITH 
liked as (SELECT Likes.ID2 as ID, count(Likes.ID2) as counter
FROM Likes
GROUP BY Likes.ID2)


SELECT Highschooler.name, Highschooler.grade
FROM Highschooler, liked
WHERE Highschooler.ID = liked.ID
AND liked.counter >= 2

 * sqlite:///Social-Network.db
Done.


name,grade
Cassandra,9
Kris,10


Social-Network Query Exercises Extras, Q1:

For every situation where student A likes student B, but student B likes a different student C, return the names and grades of
A, B, and C.

In [14]:
%%sql

WITH 
student1 as (SELECT * FROM Highschooler),
student2 as (SELECT * FROM Highschooler),
student3 as (SELECT * FROM Highschooler),
likes1 as (SELECT * FROM Likes),
likes2 as (SELECT * FROM Likes)


SELECT student1.name, student1.grade, student2.name, student2.grade, student3.name, student3.grade
FROM student1, student2, student3, likes1, likes2
WHERE student1.ID = likes1.ID1 
AND student2.ID = likes1.ID2

AND student2.ID = likes2.ID1
AND student3.ID = likes2.ID2

AND student1.ID != student2.ID
AND student2.ID != student3.ID
AND student1.ID != student3.ID

 * sqlite:///Social-Network.db
Done.


name,grade,name_1,grade_1,name_2,grade_2
Andrew,10,Cassandra,9,Gabriel,9
Gabriel,11,Alexis,11,Kris,10


Social-Network Query Exercises Extras, Q2:

Find those students for whom all of their friends are in different grades from themselves. Return the students' names and
grades.

In [15]:
%%sql

WITH 
student1 as (SELECT * FROM Highschooler),
student2 as (SELECT * FROM Highschooler),
samegrade as (SELECT DISTINCT student1.ID as ID
FROM student1, student2, Friend
WHERE student1.ID = Friend.ID1
AND student2.ID = Friend.ID2
AND student1.ID != student2.ID
AND student1.grade = student2.grade)

SELECT Highschooler.name, Highschooler.grade
FROM Highschooler
WHERE Highschooler.ID NOT IN (SELECT ID FROM samegrade)

 * sqlite:///Social-Network.db
Done.


name,grade
Austin,11


Social-Network Query Exercises Extras, Q3:

What is the average number of friends per student? (Your result should be just one number.)

In [16]:
%%sql

WITH 
students as (SELECT count(*) as counter FROM Highschooler),
friends as (SELECT count(*) as counter FROM Friend)

SELECT friends.counter * 1.0 / students.counter as average
FROM friends, students

 * sqlite:///Social-Network.db
Done.


average
2.5


Social-Network Query Exercises Extras, Q4:

Find the number of students who are either friends with Cassandra or are friends of friends of Cassandra. Do not count
Cassandra, even though technically she is a friend of a friend.

In [18]:
%%sql

WITH 
cassandra as (SELECT Highschooler.ID as ID FROM Highschooler WHERE Highschooler.name = 'Cassandra'),
cass_friend as (
SELECT Highschooler.ID
FROM Highschooler, cassandra, Friend
WHERE Highschooler.ID = Friend.ID1
AND cassandra.ID = Friend.ID2)

SELECT count(DISTINCT Friend.ID2)
FROM Friend, cassandra, cass_friend
WHERE (Friend.ID1 = cassandra.ID OR Friend.ID1 = cass_friend.ID)
AND Friend.ID2 != cassandra.ID


 * sqlite:///Social-Network.db
Done.


count(DISTINCT Friend.ID2)
7


Social-Network Query Exercises Extras, Q5:

Find the name and grade of the student(s) with the greatest number of friends.

In [19]:
%%sql

WITH
friendlist as (SELECT Friend.ID1 as ID, count(*)as friends
FROM Friend
GROUP BY Friend.ID1),

maximum as (SELECT MAX(friends) as top
FROM friendlist)

SELECT Highschooler.name, Highschooler.grade
FROM Highschooler, friendlist, maximum
WHERE Highschooler.ID = friendlist.ID
AND friendlist.friends = maximum.top
ORDER BY name

 * sqlite:///Social-Network.db
Done.


name,grade
Alexis,11
Andrew,10


Social-Network Modification Exercises, Q1:

It's time for the seniors to graduate. Remove all 12th graders from Highschooler.

In [22]:
%%sql

DELETE FROM Highschooler
WHERE Highschooler.grade = 12

 * sqlite:///Social-Network.db
(sqlite3.OperationalError) near "SELECT": syntax error
[SQL: DELETE FROM Highschooler
WHERE Highschooler.grade = 12

SELECT *
FROM Highschooler]
(Background on this error at: http://sqlalche.me/e/14/e3q8)


Social-Network Modification Exercises, Q2:

If two students A and B are friends, and A likes B but not vice-versa, remove the Likes tuple.

Social-Network Modification Exercises, Q3:

For all cases where A is friends with B, and B is friends with C, add a new friendship for the pair A and C. Do not add duplicate friendships, friendships that already exist, or friendships with oneself. _(This one is a bit challenging; congratulations if you get it right.)_